In [1]:
import sys
import ROOT as R
# R.EnableImplicitMT()
%jsroot on

In [2]:
R.gSystem.Load("/data/HPS/lib/libMoller.dylib")
Moller = R.Moller()
print("Version of Moller Class:",Moller.GetVersion())

Version of Moller Class: 0.1.2


In [3]:
data_files_pattern = "/data/HPS/data/physrun2016/Moller/rafo_minidst/hps_007966.?_Moller_4.2_minidst.root"
snapshot_file_name = "moller_rafo_filtered.root"
#data_files_pattern = "/data/HPS/data/physrun2016/Moller/pass4kf/hps_007966.0_recon_5.2.1_minidst.root"
#snapshot_file_name = "moller_pass4kf_filtered.root"
ch_moller = R.TChain("MiniDST")
nf = ch_moller.Add(data_files_pattern)
n_events = ch_moller.GetEntries()
df_moller = R.RDataFrame(ch_moller)
df = df_moller
print(f"Added {nf} files to the chain, total of {n_events/1e6:7.2f}M events")

Added 10 files to the chain, total of    0.06M events


In [4]:
Particle_names=["Final state particle KF", "UC V0 vertex KF", "BSC V0 vertex KF", "TC V0 vertex KF", "UC Moller vertex KF",
"BSC Moller vertex KF", "TC Moller vertex KF", "Other electron KF", "UC VC vertex KF", "Final state particle GBL", "UC V0 vertex GBL",
"BSC V0 vertex GBL", "TC V0 vertex GBL", "UC Moller vertex", "BSC Moller vertex", "TC Moller vertex", "Other electron GBL",
"UC VC vertex GBL"]
All_Names = list(df_moller.GetColumnNames())
Store_Names = []
ii=0
remove_names = [] # ["v0_","hodo_","ext_trigger","part_","rf_","svt"]
for n in All_Names:
    s = str(n)
    if not any([s.lower().startswith(x) for x in remove_names]):
        Store_Names.append(s)
        dat_type = df_moller.GetColumnType(s).replace("ROOT::VecOps::RVec<","RVec<")
        print(f"{s:<30s} [{dat_type:<20s}]", end = " ")
        ii += 1
        if ii%2==0:
            print("");


ecal_cluster_energy            [RVec<double>        ] ecal_cluster_hits              [RVec<vector<int>>   ] 
ecal_cluster_nhits             [RVec<int>           ] ecal_cluster_seed_energy       [RVec<double>        ] 
ecal_cluster_seed_index        [RVec<int>           ] ecal_cluster_seed_ix           [RVec<int>           ] 
ecal_cluster_seed_iy           [RVec<int>           ] ecal_cluster_time              [RVec<double>        ] 
ecal_cluster_uncor_energy      [RVec<double>        ] ecal_cluster_uncor_hits        [RVec<vector<int>>   ] 
ecal_cluster_uncor_nhits       [RVec<int>           ] ecal_cluster_uncor_seed_energy [RVec<double>        ] 
ecal_cluster_uncor_seed_index  [RVec<int>           ] ecal_cluster_uncor_seed_ix     [RVec<int>           ] 
ecal_cluster_uncor_seed_iy     [RVec<int>           ] ecal_cluster_uncor_time        [RVec<double>        ] 
ecal_cluster_uncor_x           [RVec<double>        ] ecal_cluster_uncor_y           [RVec<double>        ] 
ecal_cluster_uncor_

In [22]:
#dfx = Moller.Select_El_Pairs(R.RDF.AsRNode(df),2.)
dfx = Moller.Select_El_Pairs(R.RDF.AsRNode(df),2.,"electron_pairs")
dfxx = Moller.Anti_Select_El_Pairs(R.RDF.AsRNode(df),4,"electron_pairs")
d = [dfx, dfxx]
for i in range(len(d)):
    d[i] = Moller.Refine_El_Pairs_1(R.RDF.AsRNode(d[i]), 2.306*0.75, 2.306*1.15, "electron_pairs", "el_pairs_r1")
    d[i] = Moller.Refine_El_Pairs_2(R.RDF.AsRNode(d[i]), "el_pairs_r1", "el_pairs_r2")
    d[i] = Moller.Add_Moller_Inv_Mass(R.RDF.AsRNode(d[i]),"el_pairs_r2","moller_inv_mass")
    d[i] = d[i].Define("n_moller_pair","return electron_pairs.size();").Filter("n_moller_pair >= 1")

# Collection of indexes to ALL the electron particles and all the electron tracks
    d[i] = d[i].Define("e_part_idx","vector<int> out;for(int i=0; i<part_pdg.size(); ++i){if(part_pdg[i]==11) out.push_back(i);} return out;")\
   .Define("e_track_idx","vector<int> out;for(int i=0; i<part_pdg.size(); ++i){if(part_pdg[i]==11) out.push_back(part_track[i]);} return out;")
# Collection of indexes to the electron particles of the Moller pair
    d[i] = d[i].Define("pair_e_part_idx","std::set<int> out;for(int i=0; i<electron_pairs.size(); ++i){out.insert(electron_pairs[i].first);out.insert(electron_pairs[i].second);} return out;")

dfx, dfxx = d

In [23]:
# num = dfx.Count()
# num2 = dfx2.Count()
# print(f" Num = {num.GetValue()}   Num2 = {num2.GetValue()}")

In [24]:
 # Example - Check that the charge of electrons is -1, first with indirection, then with a direct selection on the part_pdg.
# h_tmp = dfx.Define("e_charge","vector<int> out;for(int i=0;i<e_part_idx.size();++i){out.push_back(part_charge[e_part_idx[i]]);} return out;").Histo1D(("h_tmp","e_charge",5,-2.5,2.5),"e_charge")
# h_tmp = dfx.Define("e_charge","vector<int> out;for(int i=0;i<part_charge.size();++i){if(part_pdg[i]==11) out.push_back(part_charge[i]);} return out;").Histo1D(("h_tmp","e_charge",5,-2.5,2.5),"e_charge")
h_n_part = dfx.Define("n_part","return part_pdg.size()").Histo1D(("h_n_part","Number of particles",11,-0.5,10.5),"n_part")
h_n_neg_track = dfx.Define("n_neg_track","int count=0;for(int i=0;i<track_omega.size();++i){if(track_omega[i]>0 && track_type[i]==1)count++;} return count;").Histo1D(("h_n_neg_track","Number of - tracks",11,-0.5,10.5),"n_neg_track")
h_n_electron = dfx.Define("n_electron","return e_part_idx.size()").Histo1D(("h_n_electron","Number of e-",11,-0.5,10.5),"n_electron")
h_n_moller = dfx.Define("n_moller","return electron_pairs.size()").Histo1D(("h_n_moller","Number of Moller pairs",11,-0.5,10.5),"n_moller")

h_track_time_diff = dfx.Define("track_time_diff","vector<double> out; for(auto p: electron_pairs){out.push_back(track_time[part_track[p.first]] - track_time[part_track[p.second]]);} return out;").Histo1D(("h_track_time_diff","track_time_diff",500,-10.,10.),"track_time_diff")
hx_track_time_diff = dfxx.Define("track_time_diff","vector<double> out; for(auto p: electron_pairs){out.push_back(track_time[part_track[p.first]] - track_time[part_track[p.second]]);} return out;").Histo1D(("hx_track_time_diff","track_time_diff",500,-10.,10.),"track_time_diff")

h_track_chi2 = dfx.Define("t_chi2","vector<double> out; std::set<int> tracks; for(auto p: electron_pairs){tracks.insert(part_track[p.first]); tracks.insert(part_track[p.second]);};for(auto t: tracks){out.push_back(track_chi2[t]);} return out;").Histo1D(("h_track_chi2","track_chi2",500,0.,10.),"t_chi2")
hx_track_chi2 = dfxx.Define("t_chi2","vector<double> out; std::set<int> tracks; for(auto p: electron_pairs){tracks.insert(part_track[p.first]); tracks.insert(part_track[p.second]);};for(auto t: tracks){out.push_back(track_chi2[t]);} return out;").Histo1D(("xh_track_chi2","track_chi2",500,0.,10.),"t_chi2")

h_track_nhit = dfx.Define("t_nhit","vector<int> out; std::set<int> tracks; for(auto p: electron_pairs){tracks.insert(part_track[p.first]); tracks.insert(part_track[p.second]);};for(auto t: tracks){out.push_back(track_n_hits[t]);} return out;").Histo1D(("h_track_nhit","track_nhit",15,-0.5,14.5),"t_nhit")
hx_track_nhit = dfxx.Define("t_nhit","vector<int> out; std::set<int> tracks; for(auto p: electron_pairs){tracks.insert(part_track[p.first]); tracks.insert(part_track[p.second]);};for(auto t: tracks){out.push_back(track_n_hits[t]);} return out;").Histo1D(("hx_track_nhit","track_nhit",15,-0.5,14.5),"t_nhit")


In [25]:
%jsroot on
cc0 = R.TCanvas("cc0","cc0",1200,500)
cc0.Divide(2,2)
cc0.cd(1)
h_n_moller.SetTitle("Particle counts")
h_n_moller.SetStats(0)
h_n_moller.SetLineWidth(2)
h_n_moller.SetLineColor(R.kCyan)
h_n_moller.Draw("same")
h_n_electron.SetLineWidth(3)
h_n_electron.SetLineColor(R.kBlue)
h_n_electron.Draw("same")
h_n_neg_track.SetLineColor(R.kRed)
h_n_neg_track.Draw("same")
h_n_part.SetLineWidth(1)
h_n_part.SetLineColor(R.kGreen+2)
h_n_part.Draw("same")
legend = R.TLegend(0.7,0.7,0.9,0.9)
legend.AddEntry(h_n_part.GetPtr(),"any particle","l")
legend.AddEntry(h_n_neg_track.GetPtr(),"neg. tracks","l")
legend.AddEntry(h_n_electron.GetPtr(),"electron","l")
legend.AddEntry(h_n_moller.GetPtr(),"Moller","l")
legend.Draw()
cc0.cd(2)
h_track_time_diff.SetLineWidth(1)
h_track_time_diff.SetLineColor(R.kGreen+2)
h_track_time_diff.Draw()
hx_track_time_diff.SetLineColor(R.kRed)
hx_track_time_diff.Draw("same")
cc0.cd(3)
h_track_chi2.SetLineWidth(1)
h_track_chi2.SetLineColor(R.kGreen+2)
h_track_chi2.Draw()
hx_track_chi2.SetLineColor(R.kRed)
hx_track_chi2.Draw("same")
cc0.cd(4)
h_track_nhit.SetStats(0)
h_track_nhit.SetLineColor(R.kRed)
h_track_nhit.Draw("same")
hx_track_nhit.SetStats(0)
hx_track_nhit.SetLineWidth(1)
hx_track_nhit.SetLineColor(R.kGreen+2)
hx_track_nhit.Draw("same")

cc0.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc0


In [26]:
h_v0_type = dfx.Histo1D(("h_v0_type","V0 Type",17,-0.5,16.5),"v0_type")
#("h_n_v0","n_v0",11,-0.5,10.5)
cc1 = R.TCanvas("cc1","cc1",1000,500)
#cc1.Divide(1,2)
pad1= cc1.cd(1)
pad1.SetBottomMargin(0.2)
axis = h_v0_type.GetXaxis()
axis.SetLabelSize(0.035)
for i in range (0,16):
    axis.SetBinLabel(i+1,Particle_names[i])
h_v0_type.SetStats(0)
h_v0_type.SetFillColor(R.kBlue-10)
h_v0_type.Draw()
cc1.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc1


In [27]:
h_v0_mass_UC = dfx.Define("v0_mass_UC","vector<double> out;for(int i=0;i<v0_mass.size();++i){if(v0_type[i]==13) out.push_back(v0_mass[i]);} return out").Histo1D(("h_v0_mass_UC","v0 mass UC; M_{inv} [GeV]",500,0.,0.2),"v0_mass_UC")
h_v0_mass_BSC = dfx.Define("v0_mass_BSC","vector<double> out;for(int i=0;i<v0_mass.size();++i){if(v0_type[i]==14) out.push_back(v0_mass[i]);} return out").Histo1D(("h_v0_mass_BSC","v0 mass UCM_{inv} [GeV]",500,0.,0.2),"v0_mass_BSC")
h_v0_mass_TC = dfx.Define("v0_mass_TC","vector<double> out;for(int i=0;i<v0_mass.size();++i){if(v0_type[i]==15) out.push_back(v0_mass[i]);} return out").Histo1D(("h_v0_mass_TC","v0 mass TCM_{inv} [GeV]",500,0.,0.2),"v0_mass_TC")

In [28]:
h_ee_mass = dfx.Histo1D(("h_ee_mass","EE inv mass",500,0.,0.2),"moller_inv_mass")
hx_ee_mass = dfxx.Histo1D(("hx_ee_mass","EE inv mass",500,0.,0.2),"moller_inv_mass")

In [33]:
cc2 = R.TCanvas("cc2","cc2",1000,500)
#cc2.Divide(1,2)
pad1= cc2.cd(1)
#pad1.SetLogy()
h_ee_mass.SetStats(0)
h_ee_mass.SetLineColor(R.kOrange)
h_ee_mass.SetLineWidth(2)
h_ee_mass.Draw()
hx_ee_mass.Scale(0.9)
hx_ee_mass.SetStats(1)
hx_ee_mass.SetLineColor(R.kRed)
hx_ee_mass.SetLineWidth(2)
hx_ee_mass.Draw("same")
h_v0_mass_UC.SetStats(0)
h_v0_mass_UC.SetLineWidth(1)
h_v0_mass_UC.SetLineColor(R.kGreen+2)
h_v0_mass_UC.Draw("same")
h_v0_mass_BSC.SetLineWidth(1)
h_v0_mass_BSC.SetLineColor(R.kCyan)
h_v0_mass_BSC.Draw("same")
h_v0_mass_TC.SetLineColor(R.kBlue)
h_v0_mass_TC.Draw("same")

legend = R.TLegend(0.6,0.7,0.9,0.9)
legend.AddEntry(h_v0_mass_UC.GetPtr(),"Unconstrained Moller","l")
legend.AddEntry(h_v0_mass_BSC.GetPtr(),"Beamspot contrained Moller","l")
legend.AddEntry(h_v0_mass_TC.GetPtr(),"Target constrained Moller","l")
legend.AddEntry(h_ee_mass.GetPtr(),"Moller","l")
legend.AddEntry(hx_ee_mass.GetPtr(),"Not Moller","l")
legend.Draw()

cc2.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc2


In [34]:
h_plot_iso0 = dfx.Define("trk_iso","vector<double> out; for(int i=0;i<track_isolation.size();++i){out.push_back(track_isolation[i][0]);}; return out").Histo1D(("h_plot_iso0","track isolation",13,-2.5,10.5),"trk_iso")
cc3 = R.TCanvas("cc3","cc3",1000,500)
cc3.SetLogy()
h_plot_iso0.Draw()
cc3.Draw()

In [23]:
disp = dfx.Define("ntrack","return track_isolation.size();").Define("n_iso", "vector<int> niso; for(int i=0;i<track_isolation.size();++i){niso.push_back(track_isolation[i].size());} return niso;").Define("trk_iso0","vector<double> out; for(int i=0;i<track_isolation.size();++i){if(track_type[i]>=32) out.push_back(track_isolation[i][0]);}; return out").Define("trk_iso1","vector<double> out; for(int i=0;i<track_isolation.size();++i){if(track_type[i]>=32) out.push_back(track_isolation[i][1]);}; return out").Define("trk_iso2","vector<double> out; for(int i=0;i<track_isolation.size();++i){if(track_type[i]>=32) out.push_back(track_isolation[i][2]);}; return out").Define("trk_iso3","vector<double> out; for(int i=0;i<track_isolation.size();++i){if(track_type[i]>=32) out.push_back(track_isolation[i][3]);}; return out").Display(
    ("track_type","trk_iso0","trk_iso1","track_y0","trk_iso2","trk_iso3"), 50)
disp.Print()

TypeError: Template method resolution failed:
  none of the 3 overloaded methods succeeded. Full details:
  ROOT::RDF::RResultPtr<ROOT::RDF::RDisplay> ROOT::RDF::RInterface<ROOT::Detail::RDF::RJittedFilter,void>::Display(initializer_list<string> columnList, size_t nRows = 5, size_t nMaxCollectionElements = 10) =>
    AbortSignal: abort from C++; program state was reset
  ROOT::RDF::RResultPtr<ROOT::RDF::RDisplay> ROOT::RDF::RInterface<ROOT::Detail::RDF::RJittedFilter,void>::Display(const ROOT::RDF::ColumnNames_t& columnList, size_t nRows = 5, size_t nMaxCollectionElements = 10) =>
    AbortSignal: abort from C++; program state was reset
  ROOT::RDF::RResultPtr<ROOT::RDF::RDisplay> ROOT::RDF::RInterface<ROOT::Detail::RDF::RJittedFilter,void>::Display(string_view columnNameRegexp = "", size_t nRows = 5, size_t nMaxCollectionElements = 10) =>
    TypeError: could not convert argument 1
  none of the 3 overloaded methods succeeded. Full details:
  ROOT::RDF::RResultPtr<ROOT::RDF::RDisplay>

[/Users/maurik/root/master/lib/libcppyy_backend.so] (anonymous namespace)::TExceptionHandlerImp::HandleException(int) (no debug info)
[/Users/maurik/root/master/lib/libCore.so] TUnixSystem::DispatchSignals(ESignals) (no debug info)
[/usr/lib/system/libsystem_platform.dylib] _sigtramp (no debug info)
[/usr/lib/system/libsystem_pthread.dylib] pthread_kill (no debug info)
[/usr/lib/system/libsystem_c.dylib] abort (no debug info)
[/usr/lib/libc++abi.dylib] __cxxabiv1::__aligned_malloc_with_fallback(unsigned long) (no debug info)
[/usr/lib/libc++abi.dylib] demangling_terminate_handler() (no debug info)
[/usr/lib/libobjc.A.dylib] _objc_terminate() (no debug info)
[/usr/lib/libc++abi.dylib] std::__terminate(void (*)()) (no debug info)
[/usr/lib/libc++abi.dylib] __cxa_get_exception_ptr (no debug info)
[/usr/lib/libc++abi.dylib] __cxxabiv1::exception_cleanup_func(_Unwind_Reason_Code, _Unwind_Exception*) (no debug info)
[/Users/maurik/root/master/lib/libROOTDataFrame.so] ROOT::Internal::RDF::Col

libc++abi: terminating due to uncaught exception of type std::runtime_error: Column "track_y0" is not in a dataset and is not a custom column been defined.
 *** Break *** abort
libc++abi: terminating due to uncaught exception of type std::runtime_error: Column "track_y0" is not in a dataset and is not a custom column been defined.
 *** Break *** abort
libc++abi: terminating due to uncaught exception of type std::runtime_error: Column "track_y0" is not in a dataset and is not a custom column been defined.
 *** Break *** abort
libc++abi: terminating due to uncaught exception of type std::runtime_error: Column "track_y0" is not in a dataset and is not a custom column been defined.
 *** Break *** abort
libc++abi: terminating due to uncaught exception of type std::runtime_error: Column "track_y0" is not in a dataset and is not a custom column been defined.
 *** Break *** abort
libc++abi: terminating due to uncaught exception of type std::runtime_error: Column "track_y0" is not in a dataset 